# Altermagnetic superconductors

In [1]:
import kwant
import numpy as np
from amsc import bulk_amsc_system, generate_intial_Delta, setup_gaussian_impurities
from pauli import *
from qm_tools import sparse_diag
import matplotlib as mpl
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

In [2]:
# eigsh = sla.eigsh
eigsh = sparse_diag  # MUMPS

In [3]:
periodic_bc = True

Nx = 81
Ny = 81

x_ax = np.linspace(-(Nx // 2), (Nx // 2), Nx)
y_ax = np.linspace(-(Ny // 2), (Ny // 2), Ny)
x, y = np.meshgrid(x_ax, y_ax)

t = 5.0
t_so = 0.0 * t
t_am = 0.25 * t

mu = 2.5
hz0 = 1e-6
hx0 = 0
hy0 = 0

## Initial value for Delta (will be used to determine g)
Delta_init = 1

In [4]:
vortex_positions = []
windings = []
l_core = 0  # l_core is the decay length of the order parameter (in a vortex that is the coherence length)

Delta, theta = generate_intial_Delta(
    x=x,
    y=y,
    Delta_init=Delta_init,
    vortex_positions=vortex_positions,
    windings=windings,
    l_core=l_core,
)

In [5]:
# Impurituy positions
impurity_positions = [(0, 0)]  # , (-10, 0)]
impurity_sizes = [2.2]  # Sizes
impurity_eccentricities = [0.0]  # Eccentricity
impurity_orientation = [0.0]  # Orientation


V_imp = [-19]  # Impurity potential (negative = attractive)
hx_imp = [0.0]  # Impurity field
hy_imp = [0.0]  # Impurity field
hz_imp = [0.0]  # Impurity field

In [6]:
V, hx, hy, hz = setup_gaussian_impurities(
    x=x,
    y=y,
    mu=mu,
    hx0=hx0,
    hy0=hy0,
    hz0=0.02,
    impurity_sizes=impurity_sizes,
    impurity_positions=impurity_positions,
    impurity_eccentricities=impurity_eccentricities,
    impurity_orientations=impurity_orientation,
    V_imp=V_imp,
    hx_imp=hx_imp,
    hy_imp=hy_imp,
    hz_imp=hz_imp,
)

syst, lat = bulk_amsc_system(
    Nx=Nx,
    Ny=Ny,
    t=t,
    t_so=0.0,
    t_am=t_am,
    V=V,
    Delta=Delta,
    theta=theta,
    hx=hx,
    hy=hy,
    hz=hz,
    periodic_bc=periodic_bc,
)

fsyst = syst.finalized()

ws, vs = eigsh(fsyst.hamiltonian_submatrix(sparse=True), k=4, sigma=0)

idxs = np.argsort(ws)
ws = ws[idxs]
vs = vs[:, idxs]

In [7]:
t0sz_op = kwant.operator.Density(fsyst, onsite=t0sz, sum=False)
t0s0_op = kwant.operator.Density(fsyst, onsite=t0s0, sum=False)

In [8]:
d_1 = t0sz_op(vs[:, 0])
d_2 = t0sz_op(vs[:, 1])
d_3 = t0sz_op(vs[:, 2])
d_4 = t0sz_op(vs[:, 3])

In [23]:
def get_central_region(data, size, region_size):
    """Extract the central region from a larger 2D data grid."""
    start = (size - region_size) // 2
    end = start + region_size
    return data[start:end, start:end]


# Example density data, replace these with actual data
density = d_3 + d_4  # Your density data

# Example potential function, replace this with actual potential extraction
potential_values = np.array(
    [V(i, j) for i, j in [fsyst.pos(n) for n in range(fsyst.graph.num_nodes)]]
)

# Reshape the data to a 2D grid
size = 81  # Size of the full grid
density_2d = density.reshape((size, size))
potential_values_2d = potential_values.reshape((size, size))

# Get the central 21x21 region
region_size = 51
central_density = get_central_region(density_2d, size=size, region_size=region_size)
central_potential = get_central_region(
    potential_values_2d, size=size, region_size=region_size
)

# Extract coordinates from the system
pos = np.array([fsyst.pos(i) for i in range(fsyst.graph.num_nodes)])
x, y = pos[:, 0], pos[:, 1]

# Reshape x and y for the full grid and then extract central region
x_2d = x.reshape((size, size))
y_2d = y.reshape((size, size))

# Get central 21x21 region coordinates
central_x = get_central_region(x_2d, size=size, region_size=region_size)
central_y = get_central_region(y_2d, size=size, region_size=region_size)

# Flatten the central region data for plotting
central_density_flat = central_density.flatten()
central_potential_flat = central_potential.flatten()
central_x_flat = central_x.flatten()
central_y_flat = central_y.flatten()


# Generate the colormap using Matplotlib
cmap = plt.get_cmap("RdBu_r")
num_colors = 256
new_cmap = np.array([cmap(x / (num_colors - 1)) for x in range(num_colors)])

# Modify the alpha channel to create transparency
new_cmap[:, 3] = np.abs(
    (np.arange(num_colors) - (num_colors // 2) + 0.5) / (num_colors // 2 - 0.5)
) ** (2)


# Convert to Plotly colorscale format
def rgba_to_rgb(rgba):
    return f"rgba({int(rgba[0] * 255)}, {int(rgba[1] * 255)}, {int(rgba[2] * 255)}, {int(rgba[3]*255)})"


# Create the colorscale list for Plotly
custom_colorscale = [
    [i / (num_colors - 1.0), rgba_to_rgb(new_cmap[i])] for i in range(num_colors)
]

# custom_colorscale = [[0, rgba_to_rgb(cmap(0))], [0.5, 'rgba(0, 1, 1, 0)'], [1, rgba_to_rgb(cmap(255))]]


# Extract the colormap from Matplotlib
def get_colormap_colors(cmap_name, num_colors):
    cmap = plt.get_cmap(cmap_name)
    return [mpl.colors.to_hex(cmap(i / (num_colors - 1))) for i in range(num_colors)]


# # Number of colors in the colormap
# num_colors = 256

# # Get colors from Matplotlib's red and blue colormaps
# colors = get_colormap_colors("RdBu", num_colors)

# # Define a custom color scale with transparency at the central value
# # Combine the blue and red color scales
# custom_colorscale = [
#     [0.0, colors[0]],  # Start of blue
#     [0.5, "rgba(255, 255, 255, 0)"],  # Transparent white at center
#     [1.0, colors[-1]],  # End of red
# ]

# Plot the potential and density
fig = go.Figure()

# Plot the potential as colored dots at z=0
fig.add_trace(
    go.Scatter3d(
        x=central_x_flat,
        y=central_y_flat,
        z=np.zeros_like(central_density_flat),
        mode="markers",
        marker=dict(
            size=2,  # Adjust size as needed
            color="silver",
            # color=central_potential_flat,
            # colorscale="black",
            # colorbar=dict(title='Potential'),
        ),
        name="Lattice",
    )
)


fig.add_trace(
    go.Surface(
        x=central_x.reshape((region_size, region_size)),
        y=central_y.reshape((region_size, region_size)),
        z=abs(central_density.reshape((region_size, region_size))),
        surfacecolor=np.tanh(
            central_density.reshape((region_size, region_size)) * 150
        ),  # Color based on original density
        colorscale=custom_colorscale,
        opacity=0.75,
        name="Density",
    )
)

fig.add_trace(
    go.Scatter3d(
        x=[0],
        y=[0],
        z=[0],
        mode="markers",
        marker=dict(
            size=12,  # Adjust size as needed
            color="black",
            # color=central_potential_flat,
            # colorscale="black",
            # colorbar=dict(title='Potential'),
        ),
        name="impurity",
    )
)


# # Create the cylinder and cone
# theta = np.linspace(0, 2 * np.pi, 50)

# x_0, y_0 = 5, 5
# d_cylinder = 2
# h_cylinder = 0.025
# h_cone = 0.025

# x_cylinder =d_cylinder * np.cos(theta)
# y_cylinder = d_cylinder * np.sin(theta)
# z_cylinder = np.linspace(0.1 - h_cylinder, 0.1, 50)
# theta, z_cylinder = np.meshgrid(theta, z_cylinder)

# x_cylinder = d_cylinder * np.cos(theta)
# y_cylinder = d_cylinder * np.sin(theta)

# # Parametric equations for cone
# z_cone = np.linspace(0.1 - h_cylinder - h_cone, 0.1 - h_cylinder, 50)
# theta, z_cone = np.meshgrid(theta, z_cone)
# r_cone = d_cylinder * ( z_cone - (0.1 - h_cylinder - h_cone)) / h_cone

# #((-z_cone - (0.1 - h_cylinder - h_cone)) / h_cone-1)  # Linear radius for the cone

# x_cone = r_cone * np.cos(theta)
# y_cone = r_cone * np.sin(theta)


# # Add the cylinder
# fig.add_trace(go.Surface(
#     x=x_cylinder,
#     y=y_cylinder,
#     z=z_cylinder,
#     colorscale=[[0, 'gold'], [1, 'gold']],
#     opacity=1.0,
#     name='Cylinder'
# ))

# # Add the cone
# fig.add_trace(go.Surface(
#     x=x_cone,
#     y=y_cone,
#     z=z_cone,
#     colorscale=[[0, 'gold'], [1, 'gold']],
#     opacity=1.0,
#     name='Cone'
# ))


fig.update_layout(
    scene=dict(
        camera=dict(
            eye=dict(
                x=0.20, y=0.20, z=0.55
            )  # Adjust these values to get the camera closer
        ),
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False, range=[-0.1, 0.18]),  # Set z-axis limits
    ),
    margin=dict(l=0, r=0, b=0, t=0),  # Adjust margins to your preference
)

f = go.FigureWidget(fig)

In [24]:
fig.show()

In [18]:
f

FigureWidget({
    'data': [{'marker': {'color': 'silver', 'size': 2},
              'mode': 'markers',
              'name': 'Lattice',
              'type': 'scatter3d',
              'uid': 'aee3d777-b4c9-4035-b29f-2f6565d186d5',
              'x': array([-25., -25., -25., ...,  25.,  25.,  25.]),
              'y': array([-25., -24., -23., ...,  23.,  24.,  25.]),
              'z': array([0., 0., 0., ..., 0., 0., 0.])},
             {'colorscale': [[0.0, '#67001f'], [0.5, 'rgba(255, 255, 255, 0)'],
                             [1.0, '#053061']],
              'name': 'Density',
              'opacity': 0.75,
              'surfacecolor': array([[-9.25118797e-19, -3.56124687e-06, -1.50380340e-05, ...,
                                      -1.50380340e-05, -3.56124687e-06,  6.44539133e-19],
                                     [ 3.56124687e-06,  1.16467030e-19, -8.69543988e-06, ...,
                                      -8.69543988e-06, -4.11340375e-18,  3.56124687e-06],
           

In [17]:
f.layout["scene"]["camera"]["eye"]

layout.scene.camera.Eye({
    'x': 0.2, 'y': 0.2, 'z': 0.55
})

In [12]:
pio.write_image(fig, "./imgs/Fig1-sketch.svg")

In [13]:
new_cmap[126]

NameError: name 'new_cmap' is not defined